In [27]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, TrainerCallback
import torch
import matplotlib.pyplot as plt
import numpy as np


file_path = "formatted_dialogue.txt" 

In [28]:
class LossLogger(TrainerCallback):
    def __init__(self):
        self.losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and 'loss' in logs:
            self.losses.append(logs['loss'])

    def plot_losses(self):
        plt.figure(figsize=(10, 6))
        plt.plot(np.arange(len(self.losses)), self.losses, label='Training Loss')
        plt.xlabel('Steps')
        plt.ylabel('Loss')
        plt.title('Training Loss over Time')
        plt.legend()
        plt.grid(True)
        plt.show()


In [ ]:

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


loss_logger = LossLogger()



# Load your dataset
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128,
    )

dataset = load_dataset("formatted_dialogue.txt", tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt2-text-style",
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_device_train_batch_size=1,
    save_steps=500,
    save_total_limit=2,
    logging_steps=10000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    callbacks=[loss_logger],
)


device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


trainer.train()


loss_logger.plot_losses()

/home/anton/.local/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
100,2.764300
200,2.465200
300,2.416500
400,2.435500
500,2.399300
600,2.364200
700,2.470300
800,2.316800
900,2.396400


In [ ]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")




('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json')